# What
分类任务，支持两种模式
1. Folder模式，需要输入`train`, `valid`两个测试集对应的目录。`labels.txt`，需要训练的label，里面每个类别一行。
2. List模式，需要输入`train`, `valid`两个测试集对应的训练文件，每行一个样本。`labels.txt`是可选参数，里面每个类别一行。`data_pattern`一个通用的目录，与train、val中的第一列进行拼接。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| AlexNet      | alexnet                                                      |
| VGG          | vgg11, vgg11_bn, vgg13, vgg13_bn, vgg16, vgg16_bn, vgg19_bn, vgg19 |
| ResNet       | resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2 |
| DenseNet     | densenet121, densenet169, densenet201, densenet161           |
| Inception    | googlenet, inception_v3                                      |
| SqueezeNet   | squeezenet1_0, squeezenet1_1                                 |
| ShuffleNetV2 | shufflenet_v2_x2_0, shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5 |
| MobileNet    | mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small         |
| MNASNet      | mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3              |
| ViT       | ViT, SimpleViT, Crossformer, TwinsSVT|

![](http://medai.icu/storage/attachments/2023/10/10/RHd9eH5U67VsOP8vqyNyBD5nGYREejkAKx3Jw16X.)

### List模式

在Onekey中List模式一般是采用labelme标注出来的结果，如果要使用自己的数据应用List模式，需要根据自己的实际情况对数据进行处理。

* `train.txt`，训练数据列表，中间用\t（Tab水平制表符）进行分割。
* `val.txt`，验证数据列表，中间用\t（Tab水平制表符）进行分割。
* `labels.txt`，label的集合，表明训练数据多少标签。
* `data_pattern`参数，所有数据存在的目录的公共前缀，如果`train.txt`,`val.txt`文件里面存放的是绝对路径，`data_pattern`设置为None即可。

In [1]:
import os
from onekey_algo.classification.run_classification import main as clf_main
from collections import namedtuple
from onekey_algo import get_param_in_cwd

# 设置参数
compare_settings = get_param_in_cwd('compare_settings')
train_f = get_param_in_cwd('train_file') or 'split_info/train-RND-0.txt'
val_f = get_param_in_cwd('val_file') or 'split_info/val-RND-0.txt'
labels_f = 'split_info/labels.txt'
data_pattern =  get_param_in_cwd('data_pattern')

for task_type, dp, ic in zip(compare_settings['task_types'], compare_settings['data_patterns'], compare_settings['in_channels']):
    for model_name in compare_settings['model_names']:
        params = dict(train=train_f,
                      valid=val_f,
                      labels_file=labels_f,
                      data_pattern=dp,
                      j=0,
                      max2use=None,
                      val_max2use=None,
                      batch_balance=False,
                      normalize_method='imagenet',
                      model_name=model_name,
                      vit_settings = {'patch_size': 64, 'dim': 1024, 'depth': 6, 'heads': 16, 'mlp_dim': 768},
                      gpus=[0],
                      batch_size=32,
                      epochs=get_param_in_cwd('epoch', 50),
                      init_lr=0.01,
                      optimizer='sgd',
                      retrain=None,
                      model_root=os.path.join(get_param_in_cwd('model_root', 'dl_models'), task_type),
                      add_date=False,
                      iters_start=0,
                      iters_verbose=1,
                      save_per_epoch=False,
                      in_channels=ic,
                      pretrained=False)
        # 训练模型
        Args = namedtuple("Args", params)
        clf_main(Args(**params))

[2025-04-27 10:21:30 - <frozen core.transformer_factory>:  45]	INFO	使用2通道，-([0.485, 0.456])/ ([0.229, 0.224])
[2025-04-27 10:21:30 - <frozen core.transformer_factory>:  45]	INFO	使用2通道，-([0.485, 0.456])/ ([0.229, 0.224])
[2025-04-27 10:21:30 - <frozen onekey_algo.datasets.ClassificationDataset>: 662]	INFO	WE RECOMMEND YOU USE SPECIFY dataset_name LIKE list for ListDataset OR folder for FolderDataset.
[2025-04-27 10:21:30 - <frozen onekey_algo.datasets.ClassificationDataset>:  83]	INFO	Parsing record file E:\\111thymus\\thymus_habitat\\sol8. 深度（迁移）学习-单（多）中心-多通道-万能图像融合-临床\\调试\\2_73 - 副本\\train-RND-2.txt
[2025-04-27 10:21:30 - <frozen onekey_algo.datasets.ClassificationDataset>:  86]	INFO		Checking file exists in E:\\111thymus\\thymus_habitat\\sol8. 深度（迁移）学习-单（多）中心-多通道-万能图像融合-临床\\调试\\2_73 - 副本\\train-RND-2.txt
[2025-04-27 10:21:30 - <frozen onekey_algo.datasets.ClassificationDataset>: 673]	INFO	We infer your kwargs to be <class 'onekey_algo.datasets.ClassificationDataset.ListDataset'>.
[20